# 년 데이터 -> 월/분기/반기 변환

---------------
## 0. 환경설정

In [1]:
##### 라이브러리 호출 #####
import numpy as np
import pandas as pd
import time
import glob
import pickle
import itertools
import copy

import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from sklearn.model_selection import train_test_split
from statsmodels.formula.api import ols

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# 데이터프레임 출력 옵션
pd.set_option('display.max_columns', 100)

#지수표현
pd.options.display.float_format = '{:.5f}'.format

----------
## 1. 입력값 기입

In [3]:
# data_folder : 원본데이터 위치(폴더명)
# save_folder : 변환데이터 저장 위치(폴더명)
# file_nm : 파일명
# transform_range : 데이터 저장범위 / 'all' / 'month' / 'quarter' / 'half' 중에 선택하여 입력('all' 입력 시 월/분기/반기 결과 모두 저장)
# target : 타겟변수(값 변환시키지 않을 str 타입 컬럼)
# year_colnm : 연도 컬럼명

data_folder = 'data'
save_folder = 'data/transform'
file_nm = ['merge_age_new','merge_cls_new','merge_dis_new','merge_part_age','merge_part_dis9']
transform_range = ['all','all','all','all','all']
target = [['SIDO','SCHOOL_TP'],['SIDO'],['SIDO','DIS_TYPE'],['SIDO_EDU_NM','PART_EDU_NM','PART_EDU_CD','SCHOOL_TP'],['PART_EDU_NM','PART_EDU_CD']]
year_colnm = ['BASE_YY','BASE_YY','BASE_YY','BASE_YY','BASE_YY']

In [4]:
# 파일 정보 통합
file_info = pd.DataFrame(file_nm).rename(columns = {0:'file_nm'})
file_info['transform_range'] = transform_range
file_info['target'] = target
file_info['year_colnm'] = year_colnm

----------------
## 2. 데이터 변환

In [5]:
for file_num in range(len(file_info)):

    # ------------------------------------------------------------------------------------------------------------------ #
    # 분석 정보 호출
    file_nm = file_info['file_nm'][file_num]
    transform_range = file_info['transform_range'][file_num]
    target = file_info['target'][file_num]
    year_colnm = file_info['year_colnm'][file_num]

    # 데이터 호출
    tot_data = pd.read_csv(data_folder + '/' + file_nm + '.csv', encoding = 'cp949', dtype = 'str')
    tot_data.rename(columns = {year_colnm:'YEAR'}, inplace=True)  # 연도 컬럼명 변경        
    # ------------------------------------------------------------------------------------------------------------------ #
    # target 리스트를 병합하여 하나의 컬럼('target')으로 생성 (ex. 강원,특수교육 => 강원_특수교육)
    for trg_num in range(len(target)):
        if len(target) == 1:
            tot_data['target'] = tot_data[target[trg_num]]
        else:
            if trg_num == 0:
                tot_data['target'] = tot_data[target[trg_num]]
            else:
                tot_data['target'] = tot_data['target'] + '_' + tot_data[target[trg_num]]
    # ------------------------------------------------------------------------------------------------------------------ #
    ## 데이터 형 변환(str -> float)
    # (1) target 컬럼을 제외한 나머지 컬럼들은 월값 생성을 위해 float 형식으로 변경
    str_col = ['YEAR','target'] + target
    float_col = list(set(tot_data.columns).difference(set(str_col)))

    for chg_col in float_col:
        tot_data[chg_col] = tot_data[chg_col].astype('float')

    # (2) YEAR 값은 nujuk_year 계산을 위해 int 형식으로 변경
    tot_data['YEAR'] = tot_data['YEAR'].astype('int')
    # ------------------------------------------------------------------------------------------------------------------ #

    # loop_target : 타겟 리스트
    loop_target = list(tot_data['target'])
    nujuk_yr_to_mon_data = pd.DataFrame()

    for trg in loop_target:

        data = tot_data.loc[tot_data['target'] == trg,]
        data = data.sort_values(by = 'YEAR').reset_index(drop=True)
        
        for int_colnm in float_col:
            df_1 = data[['YEAR', int_colnm]]  # 월/분기/반기로 나눌 컬럼데이터
            df_2 = data[[int_colnm]].shift(1).rename(columns = {int_colnm : str(int_colnm + '_sft')})  # 위 컬럼데이터의 shift값
            df = pd.concat([df_1, df_2], axis = 1)

            # (1/12 값) 컬럼 생성
            df['diff'] = df[int_colnm] - df[int_colnm + '_sft']
            df['diff_12'] = df['diff']/12
            df['diff_12'] = df['diff_12'].shift(-1)
            df = df[['YEAR', int_colnm, 'diff_12']]

            nujuk_month_value = []  # 월값 생성 컬럼 (df[int_colnm] + df[diff_12])
            nujuk_year = []         # 연도 컬럼
            nujuk_month = []        # 월 컬럼

            # num1 : df의 개수 (각 행(연값)값 사용)
            # num2 : month 로테이션 (각 월마다 diff_12 값 추가)
            for num1 in range(len(df)):
                for num2 in range(0,12):  # 월 컬럼 생성 for문
                    if num2 == 0:  # 첫번째 값은 그대로 입력(첫번째 연값)
                        # nujuk_month_value : 월값 생성 컬럼 (df[int_colnm] + df[diff_12])
                        nujuk_month_value.append(df[int_colnm][num1])   
                        # nujuk_year : 연도 컬럼
                        nujuk_year.append(df['YEAR'][num1])
                        # nujuk_month : 월 컬럼
                        nujuk_month.append(12)  # 연값을 월값으로 변경하는 것이므로, 첫번째 데이터는 12월 값임  
                    else:
                        if num1 != (len(df) - 1):  # num1의 for문이 마지막이 아닌 경우 1/12 값 추가
                            # nujuk_month_value : 월값 생성 컬럼
                            nujuk_month_value.append(df[int_colnm][num1] + (num2 * df['diff_12'][num1]))
                            # nujuk_year : 연도 컬럼
                            nujuk_year.append(df['YEAR'][num1] + 1)
                            # nujuk_month : 월 컬럼
                            nujuk_month.append(num2)

            df_month_value = pd.DataFrame(nujuk_month_value).rename(columns = {0:int_colnm})
            df_year = pd.DataFrame(nujuk_year).rename(columns = {0:'YEAR'})
            df_month = pd.DataFrame(nujuk_month).rename(columns = {0:'MONTH'})

            if int_colnm == float_col[0]:
                df_yr_to_mon = pd.concat([df_year, df_month, df_month_value], axis = 1)
            else:
                df_yr_to_mon = pd.concat([df_yr_to_mon, df_month_value], axis = 1)

        # 컬럼 순서 변경
        col_list = ['target'] + list(df_yr_to_mon.columns)            
        df_yr_to_mon['target'] = trg
        df_yr_to_mon = df_yr_to_mon[col_list]

        # 데이터 누적
        nujuk_yr_to_mon_data = nujuk_yr_to_mon_data.append(df_yr_to_mon)
#         print('df_yr_to_mon 건수 : ', len(df_yr_to_mon))

    ## nujuk_yr_to_mon_data : 월데이터

    # MONTH 컬럼 수정
    def fn_mody_month(x):
        if int(x) < 10:
            x = str('0' + str(x))
        return x

    nujuk_yr_to_mon_data['MONTH'] = list(map(fn_mody_month, nujuk_yr_to_mon_data['MONTH']))
    nujuk_yr_to_mon_data['MONTH'] = nujuk_yr_to_mon_data['MONTH'].astype('str')
    nujuk_yr_to_mon_data['YEAR'] = nujuk_yr_to_mon_data['YEAR'].astype('int').astype('str')

    # ****************************************************************************************************************** #
    # 연 데이터 -> 월 데이터 저장
    month_data = copy.deepcopy(nujuk_yr_to_mon_data)
    # ****************************************************************************************************************** #
    # 월 데이터 -> 분기 데이터 저장
    quarter_data = copy.deepcopy(nujuk_yr_to_mon_data)

    quarter_data.loc[quarter_data['MONTH'].isin(['01','02','04','05','07','08','10','11']),'QUARTER'] = '00'        
    quarter_data.loc[quarter_data['MONTH'].isin(['03']),'QUARTER'] = '01'
    quarter_data.loc[quarter_data['MONTH'].isin(['06']),'QUARTER'] = '02'
    quarter_data.loc[quarter_data['MONTH'].isin(['09']),'QUARTER'] = '03'
    quarter_data.loc[quarter_data['MONTH'].isin(['12']),'QUARTER'] = '04'

    # 컬럼 순서 변경
    col_list = list(quarter_data.columns)
    col_list = set(col_list).difference(('target','YEAR','MONTH','QUARTER'))
    col_list = ['target','YEAR','QUARTER'] + list(col_list)

    quarter_data = quarter_data[col_list]  # 컬럼 순서 변경
    quarter_data = quarter_data.loc[quarter_data['QUARTER'] != '00',]  # 분기 값이 아닌 경우 제외
    # ****************************************************************************************************************** #
    # 월 데이터 -> 반기 데이터 저장
    half_data = copy.deepcopy(nujuk_yr_to_mon_data)

    half_data.loc[half_data['MONTH'].isin(['01','02','03','04','05','07','08','09','10','11']),'HALF'] = '00'
    half_data.loc[half_data['MONTH'].isin(['06']),'HALF'] = '01'
    half_data.loc[half_data['MONTH'].isin(['12']),'HALF'] = '02'

    # 컬럼 순서 변경
    col_list = list(half_data.columns)
    col_list = set(col_list).difference(('target','YEAR','MONTH','HALF'))
    col_list = ['target','YEAR','HALF'] + list(col_list)

    half_data = half_data[col_list]  # 컬럼 순서 변경
    half_data = half_data.loc[half_data['HALF'] != '00',]  # 반기 값이 아닌 경우 제외
    # ****************************************************************************************************************** #
    # 연 데이터 - 새로 저장하기 위해 컬럼 정리
    tot_data = tot_data[['target','YEAR'] + list(set(tot_data.columns).difference(set(['target','YEAR'] + target)))]
    # ****************************************************************************************************************** #
    # 결과 저장
    if transform_range == 'all':
        month_data.to_csv(save_folder + '/' + 'month_' + file_nm + '.csv', index = False, encoding = 'cp949')
        quarter_data.to_csv(save_folder + '/' + 'quarter_' + file_nm + '.csv', index = False, encoding = 'cp949')
        half_data.to_csv(save_folder + '/' + 'half_' + file_nm + '.csv', index = False, encoding = 'cp949')
        tot_data.to_csv(save_folder + '/' + 'year_' + file_nm + '.csv', index = False, encoding = 'cp949')
    elif transform_range == 'month':
        month_data.to_csv(save_folder + '/' + 'month_' + file_nm + '.csv', index = False, encoding = 'cp949')
        tot_data.to_csv(save_folder + '/' + 'year_' + file_nm + '.csv', index = False, encoding = 'cp949')
    elif transform_range == 'quarter':
        quarter_data.to_csv(save_folder + '/' + 'quarter_' + file_nm + '.csv', index = False, encoding = 'cp949')
        tot_data.to_csv(save_folder + '/' + 'year_' + file_nm + '.csv', index = False, encoding = 'cp949')
    elif transform_range == 'half':
        half_data.to_csv(save_folder + '/' + 'half_' + file_nm + '.csv', index = False, encoding = 'cp949')
        tot_data.to_csv(save_folder + '/' + 'year_' + file_nm + '.csv', index = False, encoding = 'cp949')
    else:
        print('월/분기/반기 모두 저장합니다.')
        month_data.to_csv(save_folder + '/' + 'month_' + file_nm + '.csv', index = False, encoding = 'cp949')
        quarter_data.to_csv(save_folder + '/' + 'quarter_' + file_nm + '.csv', index = False, encoding = 'cp949')
        half_data.to_csv(save_folder + '/' + 'half_' + file_nm + '.csv', index = False, encoding = 'cp949')
        tot_data.to_csv(save_folder + '/' + 'year_' + file_nm + '.csv', index = False, encoding = 'cp949')
    # ****************************************************************************************************************** #

print('(월/분기/반기) 변환 종료')

(월/분기/반기) 변환 종료


--------